In [1]:
import re
import os
from collections import Counter

def get_filename(path):
    name = []
    file_name = os.listdir(path)
    for novel_name in file_name:
        name.append(path + '//' + novel_name)
    return name

name_list = get_filename(r'./data')

words = []
for path in name_list:
    with open(path, "r", encoding="ANSI") as file:
        line = [line.strip("\n").replace("\u3000", "").replace("\t", "") for line in file][3:]#去掉换行，替换空格
        words += line
# words 存储语料库，其中以每一个自然段为一个分割
regex_str = ".*?([^\u4E00-\u9FA5]).*?"#正则表达式：匹配除/n之外的中文字符
english = u'[a-zA-Z0-9’!"#$%&\'()*+,-./:：;「<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'#英文字符
symbol = []
for j in range(len(words)):
    words[j] = re.sub(english, "", words[j])#将每一段文字中的英文字符替换成空格
    symbol += re.findall(regex_str, words[j])#使用正则表达式将摘除英文字符的段落提取出中文字符
word_times = Counter(symbol)#统计词频
times_top = word_times.most_common()#频率前十的词
noise_symbol = []
for eve_tuple in times_top:
    if eve_tuple[1] < 200:#如果词频在200以下说明是噪声数据
        noise_symbol.append(eve_tuple[0])
noise_number = 0
for line in words:#替换噪声
    for noise in noise_symbol:
        line.replace(noise, "")
        noise_number += 1
print("完成的噪声数据替换点：", noise_number)
print("替换的噪声符号：")
for i in range(len(noise_symbol)):
    print(noise_symbol[i], end=" ")
    if i % 50 == 0:
        print()
with open("预处理后的文本.txt", "w", encoding="utf-8") as f:
    for line in words:
        if len(line) > 1:
            print(line, file=f)


完成的噪声数据替换点： 21276696
替换的噪声符号：
〖 
〗 — ｉ ｏ ｇ ｅ ｎ ＋ ｄ · ｓ ｃ ｂ ａ ｒ ｊ  ｕ ～ ｈ ｔ － ｌ ─ ＜ ＞ ｐ ３ ｋ Ｖ ＠ ○ ９ １ 〓 ① ② ③ Ｈ Ｃ Ｔ ④ ⑤ ｆ Ａ ｙ Ｓ ● ⑥ ⑧ 
Ｇ ｍ ４ ⑦    ┦ Ｐ ∶  Ｋ ＂    × ５ ※ ６ ⑨ Ｍ  Ｂ  ８ ｗ Ｌ ２ ０ Ｒ        ［ ］   Ｗ ｚ ｘ  ⒑    
          Γ  ＝ ⒅   Ｄ τ  ⑹ Ｙ Ｊ Ｅ Ｆ Ｎ ｖ  ⒌    ǖ ァ    Ｏ      〕   ７ δ   ソ 
     χ 〈 α ィ    Φ  ば  ⑼ ￡ ο π С   ￣ ／  ┳ チ   ɑ シ ┕  ㄅ  ┒ б  ┰        Щ γ  
   ド ≌   ζ  ち  ∑    ξ  ∝  ┨   ń      ┬      フ     ρ ＃  Α   ┢ ㈡   Ρ 
 〉 セ υ ぶ            ビ    ㄍ  ゲ        Τ ∽ ぷ     ㄊ  Ω           
  ⒍   ぃ   ジ  Ψ  ⑸         ∪  ⑾    σ ⑷  て  ┯   Ｘ Ｚ ⒃ ∩    é Ｉ   ＼  Υ ℃ 
    Ｑ 

In [2]:
import jieba
from collections import Counter
import math
with open("预处理后的文本.txt", "r", encoding="utf-8") as f:
    words = [eve.strip("\n") for eve in f]

In [5]:
# 1-gram
token = []
for para in words:
    token += jieba.lcut(para)#中文分词
token_num = len(token)
count = Counter(token)
vocab1 = count.most_common()
entropy_1gram = sum([-(eve[1]/token_num)*math.log((eve[1]/token_num),2) for eve in vocab1])#1-gram算法计算信息熵
print("总词数：", token_num, " ", "不同词的个数：", len(vocab1))
print("出现频率前10的1-gram词语：", vocab1[:10])
print("entropy_1gram:", entropy_1gram)

词库总词数： 4314424   不同词的个数： 173004
出现频率前10的1-gram词语： [('的', 115604), ('了', 104527), ('他', 64712), ('是', 64457), ('道', 58623), ('我', 57483), ('你', 56679), ('在', 43691), ('也', 32606), ('这', 32199)]
entropy_1gram: 12.1667882280937


In [ ]:
# 2-gram
def combine2gram(cutword_list):
    if len(cutword_list) == 1:
        return []
    res = []
    for i in range(len(cutword_list)-1):
        res.append(cutword_list[i] + "kk"+ cutword_list[i+1])
    return res
token_2gram = []
for para in words:
    cutword_list = jieba.lcut(para)
    token_2gram += combine2gram(cutword_list)
# 2-gram的频率统计
token_2gram_num = len(token_2gram)
ct2 = Counter(token_2gram)
vocab2 = ct2.most_common()
# print(vocab2[:20])
# 2-gram相同句首的频率统计
same_1st_word = [eve.split("kk")[0] for eve in token_2gram]
assert token_2gram_num == len(same_1st_word)
ct_1st = Counter(same_1st_word)
vocab_1st = dict(ct_1st.most_common())
entropy_2gram = 0
for eve in vocab2:
    p_xy = eve[1]/token_2gram_num
    first_word = eve[0].split("s")[0]
    # p_y = eve[1]/vocab_1st[first_word]
    entropy_2gram += -p_xy*math.log(eve[1]/vocab_1st[first_word], 2)
print("词库总词数：", token_2gram_num, " ", "不同词的个数：", len(vocab2))
print("出现频率前10的2-gram词语：", vocab2[:10])
print("entropy_2gram:", entropy_2gram)
